In [34]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import backtrader as bt
import backtrader.feeds as btfeeds
from pathlib import Path
import datetime
import csv
import math

from workspace.lib.utils import construct_csv_path, write_to_csv, merge_csv
from workspace.lib.scraper import get_candles, get_exchange

exchange_id = 'ftx'
pair = 'BTC-PERP'
timeframe = '5m'

base_path = Path('../data/')
full_path = construct_csv_path(base_path, exchange_id, pair, timeframe)

data = btfeeds.GenericCSVData(
    dataname=full_path,
    timeframe=bt.TimeFrame.Minutes, compression=5,  # for 5 min candles?
    fromdate=datetime.datetime(2021, 1, 1),
    todate=datetime.datetime(2021,2,1),
    dtformat=lambda x: datetime.datetime.utcfromtimestamp(int(x) // 1000),
    openinterest=-1
)

def _get_data(from_datetime, to_datetime):
    exchange = get_exchange(exchange_id)
    candles = get_candles(exchange, pair, timeframe, from_datetime, to_datetime, logger=print)
    return candles

def get_and_merge_data(from_datetime, to_datetime):
    candles = _get_data(from_datetime, to_datetime)
    print(candles[-1][0])
    merge_csv(full_path, candles)
    return candles

def get_and_overwrite_data(from_datetime, to_datetime):
    candles = _get_data(from_datetime, to_datetime)
    write_to_csv(full_path, candles)
    return candles


In [8]:
def run_strategy(data, strat_class, params):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strat_class, **params)
    cerebro.adddata(data)
    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(commission=0.0007)

    print('Starting portfolio value: %.2f' % (cerebro.broker.getvalue()))
    cerebro.run()
    print('Final value: %.2f' % (cerebro.broker.getvalue()))

    return cerebro  # do what you want with it eg plot

    

In [15]:
class NoopStrat(bt.Strategy):
    # Strat class that only logs and passes params
    params = (
        ('minutes', 0),
        ('logging', True)
    )
    
    def __init__(self):
        if self.params.logging:
            print('Logging every %s minutes' % self.params.minutes)
        else:
            print('Logging disabled')

    def log(self, txt, dt=None):
        if not self.params.logging:
            return
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def next(self):
        time = self.datas[0].datetime.time(0)
        if time.minute % self.params.minutes == 0:
            self.log('Close: %s' % self.datas[0].close[0])
        

In [16]:
cerebro = run_strategy(data, NoopStrat, {
    'minutes': 60,
    'logging': False
})  # Todo: limit to only certain time

Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00


## Strategy testing

Define a function that takes a strategy and runs it across several backtest timeframes sequentially

First we ensure we have enough data in our feed. Naive for now -- does not take into account different timeperiods in the year, just a blanket X days before the present

In [37]:
test_period_in_days = 31
num_periods = 8
offset_in_days = 31 * 3  # leave 3 months in front for possible walk forward analysis?

_delta = datetime.timedelta(days=test_period_in_days * num_periods + offset_in_days)
earliest_required_timestamp = datetime.datetime.now() - _delta
oldest_available_timestamp = None

# Read first line of csv and check if its earlier than the earliest required date
try:
    with full_path.open('r') as f:
        reader = csv.reader(f)
        headers = next(reader)
        row1 = next(reader)
        oldest_available_timestamp = datetime.datetime.fromtimestamp(int(row1[0]) // 1000)
except FileNotFoundError:
    print('.csv not found, querying data...')
    candles = get_and_overwrite_data(earliest_required_timestamp, datetime.datetime.now())
    oldest_available_timestamp = datetime.datetime.fromtimestamp(int(candles[-1][0]) // 1000)

if earliest_required_timestamp < oldest_available_timestamp:
    # We have insufficient data in our csv
    get_and_merge_data(earliest_required_timestamp, oldest_available_timestamp)
    

In [32]:
def run_strategy_period(data_path, from_datetime, to_datetime, strat_class, params):
    data = btfeeds.GenericCSVData(
        dataname=data_path,
        timeframe=bt.TimeFrame.Minutes, compression=5,  # for 5 min candles?
        fromdate=from_datetime,
        todate=to_datetime,
        dtformat=lambda x: datetime.datetime.utcfromtimestamp(int(x) // 1000),
        openinterest=-1
    )

    cerebro = bt.Cerebro(
        stdstats=False,
    )
    cerebro.addobserver(bt.observers.BuySell, barplot=True, bardist=0.005)
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.Trades)
    cerebro.addstrategy(strat_class, **params)
    cerebro.adddata(data)
    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(commission=0.0007)

    print('Period from %s to %s' % (from_datetime.date().isoformat(), to_datetime.date().isoformat()))
    print('Starting portfolio value: %.2f' % (cerebro.broker.getvalue()))
    result = cerebro.run()  # https://www.backtrader.com/docu/cerebro/#reference
    print('Final value: %.2f' % (cerebro.broker.getvalue()))

    return (result, cerebro)  # do what you want with it eg plot
    

In [33]:
(result, cerebro) = run_strategy_period(
    full_path, 
    datetime.datetime(2021, 1, 1),
    datetime.datetime(2021, 2, 1),
    NoopStrat,
    {
        'minutes': 60,
        'logging': False
    })

print(result)

Period from 2021-01-01 to 2021-02-01
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00


In [54]:
def run_sequential(
    data_path,
    strat_class,
    params,
    duration, 
    num_periods, 
    start_date):
    period_delta = datetime.timedelta(days=duration)
    ending_results = []
    
    for i in range(num_periods):
        from_datetime = start_date + period_delta * i
        to_datetime = from_datetime + period_delta
        (result, cerebro) = run_strategy_period(
            data_path,
            from_datetime,
            to_datetime,
            strat_class,
            params
        )
        ending_results.append(cerebro.broker.getvalue())
    
    profitable_results = []
    losing_results = []

    for res in ending_results:
        if res >= 10000:
            profitable_results.append(res)
        else:
            losing_results.append(res)
    
    return (ending_results, profitable_results, losing_results)

        
def print_results(ending_results, profitable_results, losing_results):
    initial_sum = 10000
    total_average = sum(ending_results) / len(ending_results)
    
    if len(profitable_results) != 0:
        profitable_average = sum(profitable_results) / len(profitable_results)
    else:
        profitable_average = 0
    
    if len(losing_results) != 0:
        losing_average = sum(losing_results) / len(losing_results)
    else:
        losing_average = 0

    profitable_proportion = len(profitable_results) / len(ending_results)
    losing_proportion = len(losing_results) / len(ending_results)        

    print(f'Initial portfolio value: {initial_sum}')
    print(f'Num tests: {len(ending_results)} (100%), Average result: {total_average:.2f}, '\
        f'Change: {(((total_average - initial_sum) / initial_sum) * 100):.2f}%')
    print(f'Profitable: {len(profitable_results)}, ({(profitable_proportion*100):.0f}%), ' \
        f'Average result: {profitable_average:.2f}, ' \
        f'Change: {(((profitable_average - initial_sum) / initial_sum) * 100):.2f}%')
    print(f'Losing: {len(losing_results)}, ({(losing_proportion*100):.0f}%), ' \
        f'Average result: {losing_average:.2f}, ' \
        f'Change: {(((losing_average - initial_sum) / initial_sum) * 100):.2f}%')


In [55]:
all_results = run_sequential(
    full_path,
    NoopStrat,
    {
        'minutes': 60,
        'logging': False
    },
    test_period_in_days,
    num_periods,
    earliest_required_timestamp
)
print('-------')
print_results(*all_results)  # could this be run parallel?

Period from 2020-07-05 to 2020-08-05
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2020-08-05 to 2020-09-05
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2020-09-05 to 2020-10-06
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2020-10-06 to 2020-11-06
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2020-11-06 to 2020-12-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2020-12-07 to 2021-01-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2021-01-07 to 2021-02-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
Period from 2021-02-07 to 2021-03-10
Starting portfolio value: 10000.00
Logging disabled
Final value: 10000.00
-------
Initial portfolio value: 10000
Num tests: 8 (100%), Average result: 10000.00, Change: 0.00%
Profitable: 

Now lets try this on a very naive strategy just to see if it works

In [52]:
class TestStrategy(bt.Strategy):
    params = (
        ('logging', True),
    )
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        if not self.params.logging:
            return
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        # keep a reference to the "close" line in the data[0] dataseries?
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        if not self.params.logging:
            print('Logging disabled')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/sell order submitted/accepted to/by broker- nothing to do
            return

        # Check if order has been completed. Broker can reject if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm: %2f' % 
                    (order.executed.price, 
                    order.executed.value, 
                    order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %2f' % 
                    (order.executed.price,
                    order.executed.value,
                    order.executed.comm))
                
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order canceled/margin/rejected')
        
        # write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % 
            (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the references
        self.log('Close, %.2f' % (self.dataclose[0]))
    
        # if order pending, cannot second a second one
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:

            if self.dataclose[0] < self.dataclose[-1]:
                # current close less than previous close
                if self.dataclose[-1] < self.dataclose[-2]:
                    # previous close less than previous previous close
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    availCash = self.broker.getcash()
                    currentClose = self.dataclose[0]
                    units = (availCash / currentClose) * 0.9
                    self.order = self.buy(size=units)
        else:
            #already in the market... we might sell
            if len(self) >= (self.bar_executed + 5):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.close()

In [57]:
all_results = run_sequential(
    full_path,
    TestStrategy,
    {
        'logging': False
    },
    test_period_in_days,
    num_periods,
    earliest_required_timestamp
)
print('-------')
print_results(*all_results)  # could this be run parallel?

Period from 2020-07-05 to 2020-08-05
Starting portfolio value: 10000.00
Logging disabled
Final value: 4350.56
Period from 2020-08-05 to 2020-09-05
Starting portfolio value: 10000.00
Logging disabled
Final value: 3638.17
Period from 2020-09-05 to 2020-10-06
Starting portfolio value: 10000.00
Logging disabled
Final value: 3906.33
Period from 2020-10-06 to 2020-11-06
Starting portfolio value: 10000.00
Logging disabled
Final value: 4723.27
Period from 2020-11-06 to 2020-12-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 4544.88
Period from 2020-12-07 to 2021-01-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 4607.35
Period from 2021-01-07 to 2021-02-07
Starting portfolio value: 10000.00
Logging disabled
Final value: 4175.51
Period from 2021-02-07 to 2021-03-10
Starting portfolio value: 10000.00
Logging disabled
Final value: 5832.92
-------
Initial portfolio value: 10000
Num tests: 8 (100%), Average result: 4472.37, Change: -55.28%
Profitable: 0, (0%)